## Predicting Which MLB Hitters Will be inducted into the Hall of Fame on First Ballot 

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
cnx = create_engine('postgresql://localhost/postgres')

In [3]:
cnx.table_names()

[u'halloffame', u'batting', u'appearances', u'master']

First, querying a local database for results of all players appearing on their first ballot for the HOF and merging with their classical baseball "counting stats"

In [4]:
#All HOF ballots
first_ballots=pd.read_sql_query('''SELECT a.playerid, a.yearid, a.inducted
FROM halloffame as a
INNER JOIN (SELECT playerid, MIN(yearid) as first
FROM halloffame 
WHERE category='Player'
AND votedby= 'BBWAA'
GROUP BY playerid) as b
ON a.playerid=b.playerid
AND a.yearid=b.first''',cnx)

In [5]:
#Counting Stats
batting=pd.read_sql_query('''SELECT playerid, SUM(g) as g, 
SUM(ab) as ab , SUM(r) as r, SUM(h) as h, SUM(h2b) as h2b, SUM(h3b) as h3b, 
SUM(hr) as hr, SUM(rbi) as rbi, SUM(sb) as sb, SUM(cs) as cs, SUM(bb) as bb,
SUM(so) as so, SUM(ibb) as ibb, SUM(hbp) as hbp, SUM(sh) as sh,SUM(sf) as sf,SUM(gidp) as gidp
FROM batting
GROUP BY playerid''',cnx)

In [6]:
fb_batting=pd.merge(first_ballots,batting, on='playerid')

Next, querying for positions played, and using only the highest occuring value

In [7]:
pos=pd.DataFrame(pd.read_sql_query('''SELECT * FROM appearances ''',cnx).groupby('playerid').sum()\
                 [['g_p','g_c','g_1b','g_2b','g_3b','g_ss','g_lf','g_cf','g_rf','g_dh']]\
                 .idxmax(axis=1).str.strip('g_'),columns=['pos'])

In [8]:
fb_batting=pd.merge(fb_batting,pos,left_on='playerid',right_index=True)

In [9]:
fb_batting.head()

,playerid,yearid,inducted,g,ab,r,h,h2b,h3b,hr,...,sb,cs,bb,so,ibb,hbp,sh,sf,gidp,pos
0,ramospe01,1978,N,696,703.0,76.0,109.0,9.0,3.0,15.0,...,2.0,2.0,22.0,316.0,0.0,2.0,41.0,2.0,5.0,p
1,pinielo01,1990,N,1747,5867.0,651.0,1705.0,305.0,41.0,102.0,...,32.0,41.0,368.0,541.0,82.0,31.0,35.0,61.0,209.0,lf
2,meusebo01,1937,N,1407,5475.0,826.0,1693.0,368.0,95.0,156.0,...,142.0,102.0,375.0,619.0,NaN,21.0,157.0,NaN,NaN,lf
3,boyerke01,1975,N,2034,7455.0,1104.0,2143.0,318.0,68.0,282.0,...,105.0,77.0,713.0,1017.0,97.0,20.0,17.0,63.0,185.0,3b
4,bucknbi01,1996,N,2517,9397.0,1077.0,2715.0,498.0,49.0,174.0,...,183.0,73.0,450.0,453.0,111.0,42.0,47.0,97.0,247.0,1b


Merging two more tables together to obtain their career Wins Above Replacement according to Fangraphs (WAR)

In [10]:
hitters_key=pd.read_csv('master_map.csv',usecols=['key_bbref','key_fangraphs'])

In [11]:
key_vals_merged = pd.merge(fb_batting,hitters_key,how='inner', left_on='playerid',right_on='key_bbref')

In [12]:
war_key=pd.read_csv('hitwar.csv',usecols=['WAR','playerid'])

In [13]:
hof_batting_2=pd.merge(key_vals_merged,war_key,how='left',left_on='key_fangraphs',right_on='playerid')

Now, there were three major epochs when the voting rules changes, so I will bin the year of vote into one of 3

In [14]:
def era_bin(year):
    if year==1936:
        return 1
    elif year>1936 and year <1967:
        return 2
    else:
        return 3

In [15]:
#converting to bin
hof_batting_2['first_yr']=hof_batting_2.yearid.apply(era_bin)

In [16]:
#cutting out mistake rows
hof_batting_2=hof_batting_2[hof_batting_2.g>0]

Dropping Pitchers

In [17]:
#cutting out pitchers
hof_batting_2 = hof_batting_2[hof_batting_2.pos!='p']

In [18]:
#cutting down to relevant rows
hof_batting_2=hof_batting_2[['playerid_x','key_fangraphs','first_yr','pos',
               'WAR','g','ab','r','h','h2b','h3b','hr','rbi','bb','so','inducted']]

Since several very notable star players were tainted by different baseball-related scandals,
I will add a feature to manually mark that for those players

In [19]:
#Bonds,Rose,Palmeiro,McGwire,Sheffield,Sosa
steroids=[1109.0,1011217.0,1266.0,1008559.0,114.0,302.0]

In [20]:
hof_batting_2['illegal']=hof_batting_2['key_fangraphs']

In [21]:
hof_batting_2['illegal']=hof_batting_2['illegal'].apply(lambda x: 1 if x in steroids else 0 )

Making dummy variables of the categorical features (position played and era of voting)

In [22]:
#make era bins and pos into dummy variables
hof_batting_2=pd.concat([hof_batting_2, 
           pd.get_dummies(hof_batting_2['first_yr']),
           pd.get_dummies(hof_batting_2['pos'])], 
           axis=1)

In [23]:
#cleaned out those with missing data (they were only like 9 and all fringy)
hof_batting_2=hof_batting_2.dropna(axis=0)

In [24]:
hof_batting_2.inducted=hof_batting_2.inducted.replace('Y',1).replace('N',0)

Creating Batting Average and Slugging Percentage Features

In [25]:
#add batting average
hof_batting_2['ba']=hof_batting_2.h/hof_batting_2.ab

In [26]:
#add slg%
hof_batting_2['slg']=([(4*hof_batting_2.hr+3*hof_batting_2.h3b+2*hof_batting_2.h2b+
 (hof_batting_2.h-hof_batting_2.hr-hof_batting_2.h3b-hof_batting_2.h2b))/hof_batting_2.ab])[0]

In [27]:
hof_batting_2.columns

Index([   u'playerid_x', u'key_fangraphs',      u'first_yr',           u'pos',
                 u'WAR',             u'g',            u'ab',             u'r',
                   u'h',           u'h2b',           u'h3b',            u'hr',
                 u'rbi',            u'bb',            u'so',      u'inducted',
             u'illegal',                1,                2,                3,
                  u'1b',            u'2b',            u'3b',             u'c',
                  u'cf',            u'dh',            u'lf',            u'rf',
                  u'ss',            u'ba',           u'slg'],
      dtype='object')

Now, that I have all of the features, let's take a look at the means of each feature grouped by label

In [28]:
print 'Non First Ballot Hall of Famers:'
hof_batting_2.groupby(hof_batting_2.inducted).mean().iloc[0]

Non First Ballot Hall of Famers:


first_yr       2.520231
WAR           32.348410
g           1659.591040
ab          5827.244220
r            846.213873
h           1639.144509
h2b          281.132948
h3b           58.539017
hr           144.371387
rbi          788.632948
bb           604.213873
so           658.579480
inducted       0.000000
illegal        0.008671
1              0.062139
2              0.355491
3              0.582370
1b             0.131503
2b             0.117052
3b             0.114162
c              0.135838
cf             0.122832
dh             0.014451
lf             0.124277
rf             0.127168
ss             0.112717
ba             0.277695
slg            0.416008
Name: 0, dtype: float64

In [29]:
print "First Ballot Hall of Famers:"
hof_batting_2.groupby(hof_batting_2.inducted).mean().iloc[1]

First Ballot Hall of Famers:


first_yr       2.750000
WAR           89.213889
g           2611.944444
ab          9549.583333
r           1599.083333
h           2846.055556
h2b          503.472222
h3b           93.083333
hr           373.527778
rbi         1513.555556
bb          1256.000000
so          1206.194444
illegal        0.000000
1              0.083333
2              0.083333
3              0.833333
1b             0.138889
2b             0.055556
3b             0.111111
c              0.055556
cf             0.138889
dh             0.055556
lf             0.138889
rf             0.194444
ss             0.111111
ba             0.298318
slg            0.489483
Name: 1, dtype: float64

In [30]:
print "Percent increase in feature from non-inducted to inducted:"
((hof_batting_2.groupby(hof_batting_2.inducted).mean().iloc[1]-hof_batting_2.groupby(hof_batting_2.inducted).mean().iloc[0])\
/hof_batting_2.groupby(hof_batting_2.inducted).mean().iloc[0])*100

Percent increase in feature from non-inducted to inducted:


first_yr      9.116972
WAR         175.790642
g            57.384824
ab           63.878207
r            88.969170
h            73.630546
h2b          79.086879
h3b          59.010755
hr          158.727013
rbi          91.921420
bb          107.873413
so           83.150930
illegal    -100.000000
1            34.108527
2           -76.558266
3            43.093466
1b            5.616606
2b          -52.537723
3b           -2.672293
c           -59.101655
cf           13.071895
dh          284.444444
lf           11.757106
rf           52.904040
ss           -1.424501
ba            7.426382
slg          17.662085
dtype: float64

# Now to put into models

In [31]:
X=hof_batting_2[['WAR','g','ab','r','h','h2b','h3b','hr','rbi','bb','so','ba','slg',1,2,3,'1b','2b','3b',
                'c','cf','dh','lf','ss','illegal']]

In [32]:
y=hof_batting_2.inducted

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

In [34]:
random_state=100
knn=KNeighborsClassifier()
logreg=LogisticRegression(random_state=random_state)
nb=GaussianNB()
svm=SVC(random_state=random_state)
dec=DecisionTreeClassifier(random_state=random_state)
fore=RandomForestClassifier(random_state=random_state)

In [35]:
mod_name=[]
score=[]
for model in (knn,logreg,nb,svm,dec,fore):
    mod_name.append(str(model))
    score.append(cross_val_score(model, X, y, scoring='roc_auc').mean())

In [36]:
sorted(zip(score,mod_name),reverse=True)

[(0.90404615513311171,
  "RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',\n            max_depth=None, max_features='auto', max_leaf_nodes=None,\n            min_samples_leaf=1, min_samples_split=2,\n            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,\n            oob_score=False, random_state=100, verbose=0, warm_start=False)"),
 (0.88967835706966136, 'GaussianNB()'),
 (0.86371426480122127,
  "KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',\n           metric_params=None, n_jobs=1, n_neighbors=5, p=2,\n           weights='uniform')"),
 (0.86361989459815547,
  "LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,\n          penalty='l2', random_state=100, solver='liblinear', tol=0.0001,\n          verbose=0, warm_start=False)"),
 (0.75118995336386651,
  "DecisionTreeClassifier(class_weight=None, criterion='gini', max_de

In [37]:
guerrero_vals=[59.3,2147,8155,1328,2590,477,46,449,1496,737,985,.318,.553,0,0,1,0,0,0,0,0,0,0,0,0]

In [38]:
nb.fit(X,y)

GaussianNB()

In [39]:
fore.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=100, verbose=0, warm_start=False)

In [40]:
sorted(zip(X.columns,fore.feature_importances_),key=lambda x: x[1],reverse=True)

[(u'ab', 0.15452754618785058),
 ('WAR', 0.11653095715504369),
 (u'h', 0.11017716399721522),
 (u'rbi', 0.10213467420183775),
 (u'g', 0.093292527352931032),
 (u'bb', 0.0741689462374375),
 (u'r', 0.0669232950982592),
 (u'hr', 0.058336955617866615),
 (u'so', 0.051126441256766672),
 (u'h3b', 0.041603679625623113),
 ('ba', 0.036369583232541816),
 (u'h2b', 0.028198621536099727),
 (3, 0.027468374955009804),
 ('slg', 0.017746427478186062),
 (u'1b', 0.0097691972500749784),
 (u'lf', 0.00433297225231231),
 (u'c', 0.0036818557733403687),
 (1, 0.0023942249190140263),
 (u'ss', 0.0011583871686344405),
 (u'cf', 5.8168703955082766e-05),
 (2, 0.0),
 (u'2b', 0.0),
 (u'3b', 0.0),
 (u'dh', 0.0),
 ('illegal', 0.0)]

In [41]:
fore.predict_proba(guerrero_vals)

/Users/moshelaster/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.8,  0.2]])

In [42]:
nb.predict_proba(guerrero_vals)

/Users/moshelaster/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.00217383,  0.99782617]])